# HW 2 

### Problem 1 (3 points)

The ElGamal cryptosystem is based on the complexity of calculating the discrete logarithm for large primes 𝑝. In this task we will ask you to implement your own ElGamal encryption scheme on Python.

  1. Implement function for generating keys. The function must generate a random prime number (use Fermat primality test). (1 point)
  
*Note. You can assume that all the numbers are small, for example, less than $2^{32}$.*

In [1]:
from numpy.random import randint
import numpy as np

def gcd(x, y):
    while(y):
        x, y = y, x % y
    return abs(x)


def power(a, exp, p):
    if exp == 1:
        return a % p
    elif exp % 2:
        return (power(a, exp // 2, p)**2 * a) % p
    else:
        return power(a, exp // 2, p)**2 % p


def fermat_test(n, trials):
    if n == 2:
        return True

    if n % 2 == 0:
        return False

    for i in range(trials):
        a = randint(2, n-2)

        if power(a, n-1, n) != 1:
            return False
    
    return True


def generate_prime(trials):
    is_prime = False
    while not is_prime:
        prime = randint(2**15, 2**20)
        is_prime = fermat_test(prime, trials)

    return prime


def check_root(n, p):
    if gcd(n, p) != 1:
        return False
    
    for exp in range(1, p - 1):
        exponent = power(n, exp, p)
        if exponent == 1:
            return False
    
    return True


def generate_prime_root(p):
    is_prime_root = False
    number = 2
    while not is_prime_root:
        number += 1
        is_prime_root = check_root(number, p)
    
    return number

        
def generate_key(trials=1000):
    p = generate_prime(trials)
    g = generate_prime_root(p)
    x = randint(2, p-1)
    y = power(g, x, p)

    return (p, g, y), x

2. Implement functions that realize the encryption and decryption in ElGamal protocol. (1 point)

In [4]:
def inverse(a, n):
    if a < 0:
        a += n

    p_i_2 = 0     
    p_i_1 = 1
    r_prev = n
    r_cur = a

    while r_cur != 0:
        q = r_prev // r_cur
        p_i_2, p_i_1 = p_i_1, p_i_2 - q * p_i_1 
        r_prev, r_cur = r_cur, r_prev - q * r_cur

    if r_prev > 1:
        raise Exception("Uninvertible {} by modulus {}".format(a, n))
    if p_i_2 < 0:
        p_i_2 = p_i_2 + n

    return p_i_2

def encrypt(g, y, p, M, k):
    a = power(g, k, p)
    b = (power(y, k, p) * M) % p
    return a, b

def decrypt(a, b, x, p):
    a_x = power(a, x, p)
    M = (b * inverse(a_x, p)) % p
    return M

3. Encrypte message $M=4$, using open key $(y=8,g=6,p=13)$, private key $(x=3,g=6,p=13)$ and random parameter $K=11$ (1 point)

In [6]:
M = 4
y = 8
x = 3
g = 6
p = 13
K = 11

a, b = encrypt(g, y, p, M, K)
print("Key and encrypted message are {}, {}".format(a, b))
M_decrypt = decrypt(a, b, x, p)
print("Decrypted message:", M_decrypt)

Key and encrypted message are 11, 7
Decrypted message: 4


### Problem 2 (4 points)

There are analogues of ElGamal cryptosystems, in which instead of the discrete logarithm problem in multiplicative fields, the discrete logarithm problem in groups of points of elliptic curves over finite fields is used. There is a quickly performed operation here - multiplication of an integer 𝑥 to the point 𝐴 (summing the point to itself an integer number of times). In this task, we will ask you to write python function that returns all group elements of a certain elliptic curve over a finite field and performs the points summation. 

1. Write a python function that list all points of elliptic curve $y^2=x^3-12x-6$ over $F_{17}$ (1 points)

In [2]:
def f(x):
    return (x**3 - 12 * x - 6) % 17

def find_all_points():
    list_of_points = [(np.inf, np.inf)]
    for x in range(17):
        y_2 = f(x)
        for y in range(17//2):
            if power(y, 2, 17) == y_2:
                list_of_points.append((x, y))
                if y:
                    list_of_points.append((x, -y))

    return list_of_points

In [5]:
find_all_points()

[(inf, inf),
 (1, 0),
 (3, 6),
 (3, -6),
 (5, 5),
 (5, -5),
 (6, 6),
 (6, -6),
 (7, 7),
 (7, -7),
 (8, 6),
 (8, -6)]

2. Compute the sum of any two points in group above and make sure that result lies within the same group. (0.5 point)

In [6]:
x1, y1 = 8, -6
x2, y2 = 5, 5
p = 17
a = -12
b = -6

def sum_on_curve(x1, y1, x2, y2, p):
    if x1 == np.inf and y1 == np.inf:
        return x2, y2
    elif x2 == np.inf and y2 == np.inf:
        return x1, y1
    
    assert power(y1, 2, p) == f(x1)
    assert power(y2, 2, p) == f(x2)
    
    if x1 == x2 and y1 == -y2:
        return np.inf, np.inf
    elif x1 == x2 and y1 == y2:
        m = ((3 * x1**2 + a) * inverse(2 * y1, p)) % p
    else:
        m = ((y2 - y1) * inverse(x2 - x1, p)) % p
    
    x3 = (m**2 - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p
    
    if 2*x3 > p:
        x3 = x3 - p
        
    if 2*y3 > p:
        y3 = y3 - p
        
    return x3, y3

x3, y3 = sum_on_curve(x1, y1, x2, y2, p)

assert power(y3, 2, p) == f(x3)

print("Sum of ({}, {}) and ({}, {}) is ({}, {})".format(x1, y1, x2, y2, x3, y3))

Sum of (8, -6) and (5, 5) is (8, 6)


3. For the point (6,6) define whether it generates the whole group or subgroup of points of this elliptic curve. (2 points)

In [8]:
order = 1
x, y = 6, 6
current_x, current_y = x, y
generated_points = [(current_x, current_y)]

while current_x != np.inf:
    order += 1
    current_x, current_y = sum_on_curve(current_x, current_y, x, y, p)
    generated_points.append((current_x, current_y))

print("Order of (6, 6) is:", order)

group_order = len(find_all_points())
if group_order == order:
    print("(6, 6) generates whole group of points of elliptic curve")
elif not group_order % order:
    print("(6, 6) generates subgroup of points of elliptic curve")
    print("Subgroup is:", generated_points)
else:
    print("Something went wrong")

Order of (6, 6) is: 4
(6, 6) generates subgroup of points of elliptic curve
Subgroup is: [(6, 6), (1, 0), (6, -6), (inf, inf)]


4. Compare the number of points with Hasse’s estimate $|N-(p+1)|\leq 2{\sqrt  {p}}$, where $N$ is the number of points on the curve and $p$ is the finite field order. (0.5 point)

$$|N-(p+1)|\leq 2{\sqrt  {p}} \to -2\sqrt{p} \leq N-(p+1) \leq 2\sqrt{p}$$
$$p+1-2\sqrt{p} \leq N \leq p+1+2\sqrt{p}$$

As we have p = 17, so we get estimation: $18 - 2\sqrt{17} \leq N \leq 18 + 2 \sqrt{17}$

In [9]:
N = len(find_all_points())
left_boundary = p + 1 - 2 * p**0.5
right_boundary = p + 1 + 2 * p**0.5
print("Hasse's etimate on N is {} <= N <= {}".format(left_boundary, right_boundary))
print("Actual N is:", N)

Hasse's etimate on N is 9.753788748764679 <= N <= 26.246211251235323
Actual N is: 12


As we can see, true number of elliptic curve's points were estimated truly by Hasse's estimate.

### Problem 3 (3 points)

Let us consider the finite field $\mathbb{Z}_p$ with $p=11$. A secret was shared using $(n,3)$-threshold Shamir secret sharing scheme. $4$ shares are known: $(2;9), (3;10), (6;4), (7;10)$. Reconstruct the polynomial $f(x)$ and the common secret $f(0)$.

 Note that each share is given in the form $(x_i; f(x_i))$.

Polynomial in our case looks like $f(x) = ax^2 + bx + c$, because $t = 3$. We know values of this polynomial in 4 points, but for us it is enough to know 3 any points. Thus we have system:
$$\begin{cases} 3^2a + 3b + c = 9a + 3b + c = 10 \mod 11\\
6^2a + 6b + c = 3a + 6b + c = 4 \mod 11\\
7^2a + 7b + c = 5a + 7b + c = 10 \mod 11
\end{cases}$$

if we will take points $(3;10), (6;4), (7;10)$. Let's multiply last equation by 2, so we get: $-a + 3b + 2c = -2$. Let's solve this system, using Gaussian method.
$$\begin{pmatrix}
9 & 3 & 1 &| 10\\
3 & 6 & 1 &| 4\\
-1 & 3 & 2 &|-2
\end{pmatrix} \sim \begin{pmatrix}
0 & -3 & 8 &| -8\\
0 & 4 & 7 &| -2\\
-1 & 3 & 2 &| -2
\end{pmatrix} \sim \begin{pmatrix}
0 & 0 & -6 &| -4\\
0 & 4 & 7 &| -2\\
-1 & 3 & 2 &| -2
\end{pmatrix}$$
So from last matrix, we derive:
$$-6 * c = -4 \to c = -3\mod 11$$
$$4b + 7c = -2 \to 4b = -7c - 2 \to c = c - 6 = -9 = 2 \mod 11$$
$$-a + 3b + 2c = -2 \to a = 3b + 2c + 2 = 2 \mod 11$$

Thus, the final polynomial is: $f(x) = 2x^2 + 2x + 8$. And common secret equal to $f(0) = 8 \mod 11$